## **範例：多AI代理人用於預訂酒店**

在現今節奏快速的世界裡，策劃商務出差不僅僅是預訂機票和酒店房間。它需要一種協調和效率的水平，這可能很難達成。這正是多AI代理人發揮作用的地方，革新了我們管理旅遊需求的方式。

想像一下，你有一隊智能代理人隨時為你效勞，共同合作精確而輕鬆地處理你行程的每個方面。憑藉我們先進的AI技術，我們創建了專門用於預訂服務和行程安排的代理人，確保流暢且無壓力的旅遊體驗。

這是一個基本場景。策劃商務出差時，我們需要諮詢商務旅行代理人以獲取機票資訊、酒店資訊等。透過AI代理人，我們可以建立用於預訂服務的代理人和用於行程安排的代理人進行合作，提升智慧水平。


# 初始化 Azure AI Agent 服務並從 **.env** 獲取配置信息

### **.env**

建立一個 .env 檔案

**.env** 包含 Azure AI Agent 服務的連接字串、AOAI 使用的模型，以及相應的 Google API 搜尋服務 API、ENDPOINT 等。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "您的 Azure AI Agent 服務模型部署名稱"

[**NOTE**] 您需要一個具有 100,000 速率限制（每分鐘代幣數）及 600 速率限制（每分鐘請求數）的模型

  您可以在 Microsoft Foundry - 模型和端點中獲取模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "您的 Azure AI Agent 服務專案連接字串"

  您可以在 AI Foundry 入口網站畫面的專案概覽中獲取專案連接字串。

- **SERPAPI_SEARCH_API_KEY** = "您的 SERPAPI 搜尋 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "您的 SERPAPI 搜尋 ENDPOINT"

要獲得 Azure AI Agent 服務的模型部署名稱和專案連接字串，您需要先建立 Azure AI Agent 服務。建議使用[此範本](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)直接建立（***注意：*** Azure AI Agent 服務目前只在有限區域內設置，建議您參考[此連結](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)來設定區域）

Agent 需要存取 SERPAPI。建議使用[此連結](https://serpapi.com/searches)註冊，註冊後可以獲得唯一的 API KEY 及 ENDPOINT。


# 登入 Azure 

你現在需要登入 Azure。在 VScode 中打開終端機並執行 `az login` 命令


# Setup 

To run this notebook, you will need to install the following libraries. Here is a list of the required libraries and the corresponding pip install commands:

azure-identity: 用於 Azure 認證。
requests: 用於發出 HTTP 請求。
semantic-kernel: 用於語義核心框架（假設這是一個自訂或特定的庫，您可能需要從特定來源或存儲庫安裝它）。


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# 解釋： 
import asyncio：這會匯入 asyncio 模組，該模組提供了 Python 非同步程式設計的支援。它允許你使用 async 和 await 語法撰寫併發程式碼。 
from typing import Annotated：這會從 typing 模組匯入 Annotated 類型。Annotated 用來為型別提示添加元資料，這在驗證、文件或工具等多種用途上都很有用。


In [ ]:
import asyncio,os
from typing import Annotated

# Explanation:
通過使用 from dotenv import load_dotenv 及 load_dotenv()，你可以輕鬆地在 .env 文件中管理配置設定和敏感資訊（例如 API 金鑰和數據庫 URL），將它們與你的源代碼分開，使你的應用程式更加安全且更易於配置。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 解釋：

導入語句：from azure.identity.aio import DefaultAzureCredential：這從 azure.identity.aio 模組中導入 DefaultAzureCredential 類。模組名稱中的 aio 部分表示它是為非同步操作設計的。

DefaultAzureCredential 的用途：DefaultAzureCredential 類是 Azure SDK for Python 的一部分。它提供了一種預設方式來驗證 Azure 服務。它會嘗試按照特定順序使用多種方法進行驗證，例如環境變數、託管身份和 Azure CLI 認證。

非同步操作：aio 模組表示 DefaultAzureCredential 類支援非同步操作。這意味著您可以將它與 asyncio 一起使用，以執行非阻塞的驗證請求。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Explanation:
Imports various modules and classes from the semantic_kernel package. Here's a breakdown of each import:

AgentGroupChat from semantic_kernel.agents: 此類處理與 AI 代理群組聊天相關的功能。AzureAIAgent 和 AzureAIAgentSettings 來自 semantic_kernel.agents.azure_ai

AzureAIAgent: 此類用於建立及管理使用 Azure AI 服務的 AI 代理。

AzureAIAgentSettings: 此類用於設定 AzureAIAgent 的設定。TerminationStrategy 來自 semantic_kernel.agents.strategies.termination.termination_strategy:

此類定義在特定條件下終止 AI 代理執行的策略。ChatMessageContent 來自 semantic_kernel.contents.chat_message_content:

此類用於處理聊天訊息內容。
AuthorRole 來自 semantic_kernel.contents.utils.author_role:

此類定義聊天訊息中作者的不同角色。

kernel_function 來自 semantic_kernel.functions.kernel_function_decorator: 此裝飾器用於定義 kernel 函式，即可在 semantic kernel 框架內執行的函式。
這些匯入設定了建立及管理能在群組聊天環境中互動的 AI 代理所需的元件，可能用於訂房或類似任務。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Explanation:
Next we import the CodeInterpreterTool class from the azure.ai.projects.models module. 

CodeInterpreterTool: 這個類別是 Azure AI SDK 的一部分，用於在 AI 項目的環境中解讀和執行程式碼。它提供執行程式碼片段、分析程式碼或在 AI 工作流程中整合程式碼執行的功能。
這個匯入設定了在您的專案中使用 CodeInterpreterTool 所需的組件，這對於涉及動態解讀和執行程式碼的任務可能非常有用。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Explanation: 
ApprovalTerminationStrategy 類別提供了一個特定策略，用於終止 AI 代理的操作。當代理互動歷史中的最後一條訊息包含詞語「saved」時，代理將會終止。這在某些情境下非常有用，例如當代理的任務在收到「已保存」的確認後即視為完成。定義互動方法。在預約計劃被保存後，接收到 saved 信號時可停止。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 說明：

這行程式碼透過呼叫 create() 方法，使用預設或預先定義的設定初始化一個 AzureAIAgentSettings 物件。這個設定物件（ai_agent_settings）接著可以用來設定和管理 AzureAIAgent 實例。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 解釋：
通過導入 requests 庫，你可以輕鬆地在 Python 代碼中發出 HTTP 請求並與網絡服務互動。


In [ ]:
import requests

# Explanation:
這是一個用來存放用於訪問 SERP（搜尋引擎結果頁面）API 服務的 API 密鑰的變量。API 密鑰是一個獨特的標識符，用於驗證與您的帳戶相關聯的請求。

'GOOGLE_SEARCH_API_KEY'：這是一個佔位符字串。您需要將 ''GOOGLE_SEARCH_API_KEY' 替換成您的實際 SERP API 密鑰。

目的：此行的目的是將 API 密鑰存儲在變量中，以便於用來驗證對 SERP API 服務的請求。訪問服務及執行搜索時皆需 API 密鑰。

如何取得 SERP API 密鑰：要取得 SERP API 密鑰，請按照 https://serpapi.com 的一般步驟（具體步驟可能會因所使用的特定 SERP API 服務而異）：

選擇 SERP API 服務：市面上有多種 SERP API 服務，例如 SerpAPI、Google Custom Search JSON API 等。選擇最符合您需求的服務。

註冊帳戶：

前往所選擇的 SERP API 服務網站 https://www.serpapi.com 並註冊帳戶。您可能需要提供一些基本資訊並驗證電子郵件地址。

建立 API 密鑰：

註冊後，登入帳戶並前往 API 區域或控制面板。尋找建立或產生新 API 密鑰的選項。
複製 API 密鑰：

產生 API 密鑰後，將其複製。此密鑰將用於驗證您對 SERP API 服務的請求。
替換佔位符：

在您的 .env 檔案中替換佔位符。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 解釋：
BookingPlugin 類別提供使用 Serpapi.com Google 搜尋 API 預訂酒店和航班的方法。它構建必要的參數、發送 API 請求，並處理回應以返回相關的預訂資訊。API 金鑰（SERPAPI_SEARCH_API_KEY）和端點（SERPAPI_SEARCH_ENDPOINT）用於驗證和向 Google 搜尋 API 發送請求。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 解釋：
SavePlugin 類別提供一個方法 saving_plan，用於使用 Azure AI 服務保存旅行計劃。它設置 Azure 憑證，建立 AI 代理，處理用戶輸入以生成並保存旅行計劃內容，並處理檔案保存及清理操作。該方法在成功完成後返回「Saved」。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 解釋：
此程式碼設定了 Azure AI 代理人來處理訂購航班和酒店，以及根據使用者輸入保存旅行計劃。它使用 Azure 憑證來建立和配置代理人，通過群組聊天處理使用者輸入，並確保在任務完成後進行適當清理。代理人使用特定的外掛程式（BookingPlugin 和 SavePlugin）來執行各自的任務。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：
本文件係使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們力求準確，請注意自動翻譯可能包含錯誤或不準確之處。原始文件之原文版本應被視為權威來源。對於重要資訊，建議採用專業人類翻譯。我們不對因使用本翻譯而引致之任何誤解或錯誤詮釋負責。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
